# Analyze Station3 batch

## Purposes
- Quatify and Visualize over performance space

In [ ]:
import os
from importlib import reload
import batch_utils
import pandas as pd
batch_name = 'station_3'

In [2]:
os.chdir('/home/jal21012/triangle_model')

In [4]:
metrics_vars = ["acc", "csse", "sse"]
["freq", "reg"] + metrics_vars

['freq', 'reg', 'acc', 'csse', 'sse']

In [5]:
reload(batch_utils)
t = batch_utils.TarabanParser(batch_name)
t.parse_all()
t.df_to_db()

100%|██████████| 36/36 [00:17<00:00,  2.01it/s]


In [12]:
c = t.get_conn()

In [14]:
query = """
SELECT * FROM TARABAN
"""

In [16]:
taraban = pd.read_sql(query, con=c)

In [17]:
taraban

,code_name,epoch,testset,task,output_name,timetick,cond,freq,reg,acc,...,output_ticks,zero_error_radius,save_freq,wf_compression,wf_clip_low,wf_clip_high,batch_name,total_sample,batch_unique_setting_string,uuid
0,station_3_r0000,1,taraban,triangle,pho,0,High-frequency exception,High,Exception,0.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size1_learning_rate0.0001,6aab4bb76c7d4d269cc0e307eb1bc66b
1,station_3_r0000,1,taraban,triangle,pho,0,High-frequency regular-inconsistent,Low,Exception,0.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size1_learning_rate0.0001,6aab4bb76c7d4d269cc0e307eb1bc66b
2,station_3_r0000,1,taraban,triangle,pho,0,Low-frequency exception,Low,Exception,0.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size1_learning_rate0.0001,6aab4bb76c7d4d269cc0e307eb1bc66b
3,station_3_r0000,1,taraban,triangle,pho,0,Low-frequency regular-inconsistent,Low,Exception,0.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size1_learning_rate0.0001,6aab4bb76c7d4d269cc0e307eb1bc66b
4,station_3_r0000,1,taraban,triangle,pho,0,Regular control for High-frequency exception,High,Regular,0.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size1_learning_rate0.0001,6aab4bb76c7d4d269cc0e307eb1bc66b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539131,station_3_r0035,1000,taraban,triangle,sem,12,Low-frequency regular-inconsistent,Low,Exception,1.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size64_learning_rate0.01,d0212cce04cb40d888b0e25164d3d79d
539132,station_3_r0035,1000,taraban,triangle,sem,12,Regular control for High-frequency exception,High,Regular,0.956522,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size64_learning_rate0.01,d0212cce04cb40d888b0e25164d3d79d
539133,station_3_r0035,1000,taraban,triangle,sem,12,Regular control for High-frequency regular-inc...,Low,Regular,1.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size64_learning_rate0.01,d0212cce04cb40d888b0e25164d3d79d
539134,station_3_r0035,1000,taraban,triangle,sem,12,Regular control for Low-frequency exception,Low,Regular,1.000000,...,13,0.1,20,log,0,999999999,station_3,10000000,batch_size64_learning_rate0.01,d0212cce04cb40d888b0e25164d3d79d


In [ ]:
# l = LexicalityParser(batch_name)
# l.parse_all()
# l.save_df_to_csv()

In [ ]:
i = ImageabilityParser(batch_name)
i.parse_all()
